In [4]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [5]:
# load ascii text and covert to lowercase
filename = "FleursDuMal.txt"
raw_text = open(filename, encoding="utf8").read()
raw_text = raw_text.lower()

Must convert characters to integer

In [6]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [7]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  144941
Total Vocab:  56


In [8]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  144841


In [9]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [10]:
# define the LSTM model
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0725 15:01:39.528229 16296 deprecation_wrapper.py:119] From E:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0725 15:01:39.564133 16296 deprecation_wrapper.py:119] From E:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0725 15:01:39.567124 16296 deprecation_wrapper.py:119] From E:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0725 15:01:40.422834 16296 deprecation_wrapper.py:119] From E:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0725 15:01:40.429815 16296 deprecation.py:506] From E:\Anaconda\lib\site-packages\keras\backend\t

In [11]:
# define the checkpoint
filepath="weights-FdM-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
#model.load_weights('weights-conan-338-0.3525.hdf5')
model.fit(X, y, epochs=10000, batch_size=64, callbacks=callbacks_list)

W0725 11:32:39.157090  3436 deprecation.py:323] From E:\Anaconda\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10000
  1152/144841 [..............................] - ETA: 2:27:50 - loss: 3.3918

In [50]:
# load the network weights
filename = "weights-FdM-29-0.4822.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [51]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [57]:
import sys

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
#print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
ujours!mêchant!
  a travers la tempête et ses convulsateurs,

  et lorsque j'entrevo d'un voleat plaintif
  et condainenent leurs prunelles mystiques.




  la priere d'un païen


  la terre est un mal! pauvre chacun étrange
  pleure des sertenints oubliés dans une île,
  aux oaris f'amour dont vos grands charmants,

  ruelle est le don jruels, comme un oiseau, volbéere,
  de l'antique vénus le plaisir s'exile.

  un ange, imprueux, soujours et forte amante,
  en tes contorsions, risibles au fond du coeur!

  tachet toujours frais qu'il ne faut pas uer yeux,
   celle-î-dans les coulerrs où l'eau dordertent leur eis,
  comme un ast plus qui font les peris liquides.

  les plus riches vieillards et lusqu'à tes noires,
  et la candeur unie à la lubricité
  donnait un charme nenchant domme un fleuve,

     et constant jusques à la tête,
  un ast pour chauffer tes deux pieds vieux où,
  et la candeur unie à la lubricité
  donnait un charme nenchant domme un fleuve,
      et constant j